# Lesson 12: Database — Storing Data

## Why Do We Need a Database?

In previous lessons, when an agent created an article, the result just appeared on screen and **disappeared**. If you close the program, everything is lost.

In practice, we need to:
- **Store** articles that have been created
- **Track** status (processing, completed, error...)
- **Search** and **filter** articles by criteria
- **Review** edit history

A **database** handles all of this.

## SQLite — The Simplest Database

SQLite is **built into** Python — no installation needed. Data is saved to a **single file** (e.g., `workspace.db`). Perfect for small to medium applications.

Comparison:
- **Without database**: Data lost when program closes
- **With SQLite**: Data saved to a file, accessible any time

In [ ]:
import sqlite3

# Create an in-memory database (disappears when done)
# Used for experimentation — doesn't affect real files
conn = sqlite3.connect(":memory:")
conn.row_factory = sqlite3.Row

# Create a table — like creating a spreadsheet
conn.execute("""
    CREATE TABLE articles (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        topic TEXT NOT NULL,
        status TEXT DEFAULT 'queued'
    )
""")

# Insert data
conn.execute("INSERT INTO articles (topic) VALUES (?)", ("SEO Guide 2026",))
conn.execute("INSERT INTO articles (topic, status) VALUES (?, ?)", ("Content Marketing", "review"))
conn.commit()

# Read data
rows = conn.execute("SELECT * FROM articles").fetchall()
for row in rows:
    print(f"#{row['id']}: {row['topic']} ({row['status']})")
conn.close()

## Basic SQL

SQL (Structured Query Language) is the language for working with databases. You only need 4 main commands:

| Command | Purpose | Example |
|---------|---------|--------|
| `CREATE TABLE` | Create a new table | `CREATE TABLE articles (id INTEGER, topic TEXT)` |
| `INSERT INTO` | Add data | `INSERT INTO articles (topic) VALUES ('SEO Guide')` |
| `SELECT` | Read/search data | `SELECT * FROM articles WHERE status = 'review'` |
| `UPDATE` | Update data | `UPDATE articles SET status = 'done' WHERE id = 1` |

### Column types:
- `INTEGER PRIMARY KEY AUTOINCREMENT` — Auto-incrementing ID number
- `TEXT NOT NULL` — Required text string
- `TEXT DEFAULT 'queued'` — Text with a default value

### What's the `?` placeholder?
When passing data to SQL, use `?` instead of concatenating strings directly. This is the **safe** way to prevent errors and security issues:
```python
# Correct:
conn.execute("SELECT * FROM articles WHERE status = ?", ("review",))

# WRONG — never do this:
conn.execute(f"SELECT * FROM articles WHERE status = '{status}'")
```

In [ ]:
conn = sqlite3.connect(":memory:")
conn.row_factory = sqlite3.Row

conn.execute("CREATE TABLE articles (id INTEGER PRIMARY KEY AUTOINCREMENT, topic TEXT, status TEXT DEFAULT 'queued', word_count INTEGER)")
conn.execute("INSERT INTO articles (topic, status, word_count) VALUES (?, ?, ?)", ("SEO Guide", "review", 2000))
conn.execute("INSERT INTO articles (topic, status, word_count) VALUES (?, ?, ?)", ("Content Tips", "queued", None))
conn.execute("INSERT INTO articles (topic, status, word_count) VALUES (?, ?, ?)", ("Link Building", "error", None))
conn.commit()

# Filter by status (WHERE)
print("Articles in review:")
for row in conn.execute("SELECT * FROM articles WHERE status = ?", ("review",)).fetchall():
    print(f"  #{row['id']}: {row['topic']} ({row['word_count']} words)")

# Update (UPDATE)
conn.execute("UPDATE articles SET status = ?, word_count = ? WHERE id = ?", ("review", 1500, 2))
conn.commit()

print("\nAll articles after update:")
for row in conn.execute("SELECT * FROM articles ORDER BY id").fetchall():
    print(f"  #{row['id']}: {row['topic']} - {row['status']} ({row['word_count'] or '-'} words)")
conn.close()

## The Real Product Database

The examples above cover basic SQL. Now let's see how the **real database** in our project works.

The `db.py` file provides these functions:
- `init_db()` — Create tables if they don't exist
- `create_article(topic, keywords)` — Create a new article (status = 'queued')
- `get_article(id)` — View details of one article
- `list_articles()` — View all articles
- `update_article_status(id, status, **fields)` — Update status and content

The pipeline calls these functions to track progress:
```
queued -> researching -> outlining -> writing -> enriching -> review
```

In [ ]:
import sys, os
sys.path.insert(0, os.path.abspath("../../output"))

from dotenv import load_dotenv
load_dotenv()

from db import init_db, create_article, get_article, list_articles, update_article_status

init_db()

# Create a test article
article_id = create_article("Test Article from Notebook", target_keywords=["test", "notebook"])
print(f"Created article #{article_id}")

# Read it back
article = get_article(article_id)
print(f"\nArticle details:")
print(f"  Topic: {article['topic']}")
print(f"  Status: {article['status']}")
print(f"  Created: {article['created_at']}")

# Update status
update_article_status(article_id, "review", word_count=1000, article_markdown="# Test\n\nThis is a test.")
article = get_article(article_id)
print(f"\nAfter update:")
print(f"  Status: {article['status']}")
print(f"  Words: {article['word_count']}")

In [ ]:
# View all articles in the database
articles = list_articles()
print(f"Total articles: {len(articles)}\n")
for a in articles:
    print(f"  #{a['id']}: {a['topic']} ({a['status']})")

## Summary

- A **database** stores data persistently — it survives program restarts
- **SQLite** is built into Python, saves to a single file
- **4 basic SQL commands**: `CREATE TABLE`, `INSERT`, `SELECT`, `UPDATE`
- **`db.py`** is the database layer of our product — provides functions to create, read, and update articles
- **Status tracking**: the database tracks each article through the pipeline (`queued` -> `researching` -> `writing` -> `enriching` -> `review`)
- Thanks to this tracking, you always know where an article is in the process

## Exercise

Using the in-memory database pattern from earlier in this lesson, write code that:

1. Creates a table called `keywords` with columns: `id` (integer, auto-increment), `keyword` (text), `search_volume` (integer)
2. Inserts 3 keywords of your choice with made-up search volumes
3. Queries for all keywords with search volume above 1000
4. Prints the results

This is the same pattern `db.py` uses — you're practicing the foundation.

In [ ]:
# Exercise: Write your code here
